<a href="https://colab.research.google.com/github/dariashcherbakovaaa/Algorithms-for-massive-data/blob/main/Algorithms_for_massive_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment settings & libraries

In [1]:
import os
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName('PageRank').getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # Property used to format output tables better
spark
import pyspark
type(spark)

sc = spark.sparkContext
spark

from google.colab import userdata
os.environ['KAGGLE_USERNAME'] = userdata.get("KAGGLE_USERNAME")
os.environ['KAGGLE_KEY'] = userdata.get("KAGGLE_KEY")

In [ ]:
import pandas as pd
import networkx as nx
from collections import defaultdict

In [ ]:
# LinkedIn dataset (Finding similar items, Market-basket analysis)
# !kaggle datasets download -d asaniczka/1-3m-linkedin-jobs-and-skills-2024
# !unzip 1-3m-linkedin-jobs-and-skills-2024.zip -d job_skills

# *The task is to implement a ranking system based on the PageRank index using the «Prado Museum Pictures» dataset, published on Kaggle under the MIT license. The entities to be ranked are pictures, linked together if they share at least a common tag in the work_tag attribute of the  prado.csv file, although you can experiment with different strategies in order to define links between pictures.*

Dear professor, this code on dowloading and unzipping data is pretty slow (~10-20 mins), im sorry. 🥺👉👈

In [2]:
# Link analysis (PageRank)
!kaggle datasets download -d maparla/prado-museum-pictures -f prado.csv -p /content/
!unzip prado-museum-pictures.zip -d prada

 27% 5.00M/18.3M [00:00<00:00, 50.2MB/s]
100% 18.3M/18.3M [00:00<00:00, 101MB/s] 
Archive:  prado-museum-pictures.zip
replace prada/images/images/._000de341-acd0-4338-b6db-ce4127fa5b37.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: No
No
No


-- just to check how it should be seen

In [ ]:
df = pd.read_csv('/content/prada/prado.csv')
df.head(3)

In [ ]:
prado_museum = spark.read.format('csv') \
    .option('header', 'true') \
    .option('inferSchema', 'true') \
    .option('path', '/content/prada/prado.csv') \
    .load()

prado_museum.show(3)

In [ ]:
prado_museum.select('work_url').show(10)

In [ ]:
prado_museum.printSchema()

In [ ]:
prado = prado_museum.select(['work_url', 'work_tags'])
prado

In [10]:
df[['work_url', 'work_tags']]

,work_url,work_tags
0,https://www.museodelprado.es/coleccion/obra-de...,Serie de dibujos para el fresco la Caída de lo...
1,https://www.museodelprado.es/coleccion/obra-de...,Sanguina;Papel agarbanzado;Desnudo femenino;Pr...
2,https://www.museodelprado.es/coleccion/obra-de...,Óleo;Tabla;Retrato civil;Caléndula / Maravilla...
3,https://www.museodelprado.es/coleccion/obra-de...,Óleo;Lienzo;Pescador/es;Peces;1785;González Ve...
4,https://www.museodelprado.es/coleccion/obra-de...,Óleo;Cartón;Escena mortuoria;Retrato de artist...
...,...,...
13482,https://www.museodelprado.es/coleccion/obra-de...,Aguafuerte;Buril;Estampa iluminada;Papel verju...
13483,https://www.museodelprado.es/coleccion/obra-de...,Entalladura [madera a la fibra];Papel japonés;...
13484,https://www.museodelprado.es/coleccion/obra-de...,"Aguada de pigmentos opacos [gouache, témpera];..."
13485,https://www.museodelprado.es/coleccion/obra-de...,Buñol y alrededores;Gelatina / Colodión;Placa ...


IDK how to fix it, so let's read it through pandas and then switch to PySpark

In [11]:
df["work_id"] = df['work_image_url'].apply(lambda x: x.split('/')[-1])

for _, row in df.iterrows():
    image_id = row["work_id"]
    image_path = os.path.join("images", image_id)

df

,work_url,work_image_url,author,author_bio,author_url,author_id,work_title,work_subtitle,work_exposed,work_description,...,inventory,expositions,ubication,technical_sheet_autores,technical_sheet_edicion_/_estado,technical_sheet_materia,technical_sheet_ceca,technical_sheet_autora,technical_sheet_lugar_de_produccion,work_id
0,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Bayeu y Subías, Francisco","Zaragoza, 1734 - Madrid, 1795\n\nSu formación ...",https://www.museodelprado.es/coleccion/artista...,b218fee4-053b-4656-8577-9aa001ad1989,Cabeza de Gigante,"1764. Lápiz negro sobre papel verdoso, 572 x 3...",No expuesto,"De tres cuartos, mirando hacia la derecha, en ...",...,"Inv. Real Museo, Sección Dibujos, 1857.\nNúm. ...",[… + escudo] (Sello: Dirección del Museo Nacio...,Dibujos de Francisco Bayeu\n ...,NaN,NaN,NaN,NaN,NaN,NaN,404387d6-a52c-4477-b598-de2a2d5a3d55.jpg
1,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Crespi, Giuseppe Maria","Bolonia, 1665 - Bolonia, 1747\n\nDiscípulo de ...",https://www.museodelprado.es/coleccion/artista...,b20a5ee9-2ebd-4e60-aadc-e91e7867cb12,Susana y los viejos,"Siglo XVIII. Sanguina sobre papel agarbanzado,...",No expuesto,"A la derecha, Susana, sentada en una fuente de...",...,"Inv. Legado Pedro Fernández Durán, 1931.\nNúm....",Veneziano\nReverso del segundo soporte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,589ee4a3-28fa-4977-a84d-7326f5c9aeb3.jpg
2,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Cronenburch, Adriaen van","Schagen (Países Bajos), 1520 - Bergum (Países ...",https://www.museodelprado.es/coleccion/artista...,26861819-ff88-4fde-8a37-56db9e1c1ba4,Dama con una flor amarilla,"Hacia 1567. Óleo sobre tabla, 107 x 79 cm",No expuesto,"Esta obra, junto a sus compañeras (P02074, P02...",...,"Catálogo Museo del Prado, 1873-1907.\nNúm. 130...","Aaaa[""a mas tres(dri)aes"", jeroglífico de ""Adr...",El retrato del Renacimiento\n ...,NaN,NaN,NaN,NaN,NaN,NaN,4a8bab74-ca91-450a-b5b7-39dd61e2d7f3.jpg
3,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"González Velázquez, Zacarías","Madrid, 1763 - Madrid, 1834\n\nZacarías Joaquí...",https://www.museodelprado.es/coleccion/artista...,a8c659ad-d887-4703-8af3-1832dfc88eb7,"Dos pescadores, uno con caña y otro sentado","1785. Óleo sobre lienzo, 174 x 135 cm",Depósito en otra institución,Forma parte de un conjunto de cartones para lo...,...,Inv. Cartones para Tapices.\nNúm. 5710.\n\n571...,5710\nManuscrito en color anaranjado.\nAnverso...,Madrid - Cuartel General del Ejército (Depósito),NaN,NaN,NaN,NaN,NaN,NaN,9af5b176-b4d3-4930-854b-5b5f252829f1.jpg
4,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Obra copiada de Cano de la Peña, Eduardo","Madrid, 1823 - Sevilla, 1897\n\nSiendo niño se...",https://www.museodelprado.es/coleccion/artista...,521b82d6-6848-4f7d-96dc-3b8f102dd8b5,Tintoretto pintando a su hija muerta (copia),"Hacia 1856. Óleo sobre cartón, 19 x 24,5 cm",No expuesto,"Marietta (1560-1590), hija mayor del pintor Ja...",...,Inv. Nuevas Adquisiciones (iniciado en 1856).\...,"Recuerdo de la esposicion, dedicado a D. Luis ...",Caballete,NaN,NaN,NaN,NaN,NaN,NaN,4c494f0a-d5ae-45ca-826b-59f4b5fd4398.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13482,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Atribuido Falck, Jeremías","Gdansk/Danzig (Polonia), 1609 - Gdansk/Danzig ...",https://www.museodelprado.es/coleccion/artista...,d351c3c1-e89b-49cf-900d-24085ed1908f,Vista del monasterio de El Escorial,"1662 - 1672. Aguafuerte, Buril, Estampa ilumin...",No expuesto,"Cervera Vera (Escorial en BN no C20-a, Real Bi...",...,Inv. Colección Cervelló.\nNúm. 166201.\n@@@\nI...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c62f7f3e-3ad3-4d

In [19]:
df.drop_duplicates()

,work_url,work_image_url,author,author_bio,author_url,author_id,work_title,work_subtitle,work_exposed,work_description,...,inventory,expositions,ubication,technical_sheet_autores,technical_sheet_edicion_/_estado,technical_sheet_materia,technical_sheet_ceca,technical_sheet_autora,technical_sheet_lugar_de_produccion,work_id
0,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Bayeu y Subías, Francisco","Zaragoza, 1734 - Madrid, 1795\n\nSu formación ...",https://www.museodelprado.es/coleccion/artista...,b218fee4-053b-4656-8577-9aa001ad1989,Cabeza de Gigante,"1764. Lápiz negro sobre papel verdoso, 572 x 3...",No expuesto,"De tres cuartos, mirando hacia la derecha, en ...",...,"Inv. Real Museo, Sección Dibujos, 1857.\nNúm. ...",[… + escudo] (Sello: Dirección del Museo Nacio...,Dibujos de Francisco Bayeu\n ...,NaN,NaN,NaN,NaN,NaN,NaN,404387d6-a52c-4477-b598-de2a2d5a3d55.jpg
1,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Crespi, Giuseppe Maria","Bolonia, 1665 - Bolonia, 1747\n\nDiscípulo de ...",https://www.museodelprado.es/coleccion/artista...,b20a5ee9-2ebd-4e60-aadc-e91e7867cb12,Susana y los viejos,"Siglo XVIII. Sanguina sobre papel agarbanzado,...",No expuesto,"A la derecha, Susana, sentada en una fuente de...",...,"Inv. Legado Pedro Fernández Durán, 1931.\nNúm....",Veneziano\nReverso del segundo soporte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,589ee4a3-28fa-4977-a84d-7326f5c9aeb3.jpg
2,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Cronenburch, Adriaen van","Schagen (Países Bajos), 1520 - Bergum (Países ...",https://www.museodelprado.es/coleccion/artista...,26861819-ff88-4fde-8a37-56db9e1c1ba4,Dama con una flor amarilla,"Hacia 1567. Óleo sobre tabla, 107 x 79 cm",No expuesto,"Esta obra, junto a sus compañeras (P02074, P02...",...,"Catálogo Museo del Prado, 1873-1907.\nNúm. 130...","Aaaa[""a mas tres(dri)aes"", jeroglífico de ""Adr...",El retrato del Renacimiento\n ...,NaN,NaN,NaN,NaN,NaN,NaN,4a8bab74-ca91-450a-b5b7-39dd61e2d7f3.jpg
3,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"González Velázquez, Zacarías","Madrid, 1763 - Madrid, 1834\n\nZacarías Joaquí...",https://www.museodelprado.es/coleccion/artista...,a8c659ad-d887-4703-8af3-1832dfc88eb7,"Dos pescadores, uno con caña y otro sentado","1785. Óleo sobre lienzo, 174 x 135 cm",Depósito en otra institución,Forma parte de un conjunto de cartones para lo...,...,Inv. Cartones para Tapices.\nNúm. 5710.\n\n571...,5710\nManuscrito en color anaranjado.\nAnverso...,Madrid - Cuartel General del Ejército (Depósito),NaN,NaN,NaN,NaN,NaN,NaN,9af5b176-b4d3-4930-854b-5b5f252829f1.jpg
4,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Obra copiada de Cano de la Peña, Eduardo","Madrid, 1823 - Sevilla, 1897\n\nSiendo niño se...",https://www.museodelprado.es/coleccion/artista...,521b82d6-6848-4f7d-96dc-3b8f102dd8b5,Tintoretto pintando a su hija muerta (copia),"Hacia 1856. Óleo sobre cartón, 19 x 24,5 cm",No expuesto,"Marietta (1560-1590), hija mayor del pintor Ja...",...,Inv. Nuevas Adquisiciones (iniciado en 1856).\...,"Recuerdo de la esposicion, dedicado a D. Luis ...",Caballete,NaN,NaN,NaN,NaN,NaN,NaN,4c494f0a-d5ae-45ca-826b-59f4b5fd4398.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13482,https://www.museodelprado.es/coleccion/obra-de...,https://content3.cdnprado.net/imagenes/Documen...,"Atribuido Falck, Jeremías","Gdansk/Danzig (Polonia), 1609 - Gdansk/Danzig ...",https://www.museodelprado.es/coleccion/artista...,d351c3c1-e89b-49cf-900d-24085ed1908f,Vista del monasterio de El Escorial,"1662 - 1672. Aguafuerte, Buril, Estampa ilumin...",No expuesto,"Cervera Vera (Escorial en BN no C20-a, Real Bi...",...,Inv. Colección Cervelló.\nNúm. 166201.\n@@@\nI...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,c62f7f3e-3ad3-4d

In [35]:
prada = df[['work_id', 'work_tags']]
prada.head(10)

,work_id,work_tags
0,404387d6-a52c-4477-b598-de2a2d5a3d55.jpg,Serie de dibujos para el fresco la Caída de lo...
1,589ee4a3-28fa-4977-a84d-7326f5c9aeb3.jpg,Sanguina;Papel agarbanzado;Desnudo femenino;Pr...
2,4a8bab74-ca91-450a-b5b7-39dd61e2d7f3.jpg,Óleo;Tabla;Retrato civil;Caléndula / Maravilla...
3,9af5b176-b4d3-4930-854b-5b5f252829f1.jpg,Óleo;Lienzo;Pescador/es;Peces;1785;González Ve...
4,4c494f0a-d5ae-45ca-826b-59f4b5fd4398.jpg,Óleo;Cartón;Escena mortuoria;Retrato de artist...
5,a6694f04-a269-4d86-8936-06ca81a7c5eb.jpg,Proyecto arquitectónico de la primera iglesia ...
6,d0567a3d-e249-4faa-9756-fc9d9a738056.jpg,Aguada;Albayalde;Pluma;Papel;Guerra / Combate;...
7,852368fd-f744-4388-b902-f01ce0bd09b4.jpg,Óleo;Lienzo;Marina;Juegos y diversiones;Pescad...
8,a173ac46-5a43-47ba-b503-5fb8136ec593.jpg,Óleo;Lienzo;Escena historicista;Historia Medie...
9,32aec933-7f5c-4c34-a90b-0a1901e14091.jpg,Cuaderno de Burdeos II o Cuaderno H;Lápiz;Pape...


In [36]:
prada = prada.assign(work_tags=prada['work_tags'].str.split(';')).explode('work_tags')
prada.head(5)

                                    work_id  \
0  404387d6-a52c-4477-b598-de2a2d5a3d55.jpg   
0  404387d6-a52c-4477-b598-de2a2d5a3d55.jpg   
0  404387d6-a52c-4477-b598-de2a2d5a3d55.jpg   
0  404387d6-a52c-4477-b598-de2a2d5a3d55.jpg   
0  404387d6-a52c-4477-b598-de2a2d5a3d55.jpg   

                                           work_tags  
0  Serie de dibujos para el fresco la Caída de lo...  
0                                        Lápiz negro  
0                                      Papel verdoso  
0                                  Estudio de cabeza  
0                                          Mitología  


In [43]:
prada.head(10)

,work_id,work_tags
0,404387d6-a52c-4477-b598-de2a2d5a3d55.jpg,Serie de dibujos para el fresco la Caída de lo...
0,404387d6-a52c-4477-b598-de2a2d5a3d55.jpg,Lápiz negro
0,404387d6-a52c-4477-b598-de2a2d5a3d55.jpg,Papel verdoso
0,404387d6-a52c-4477-b598-de2a2d5a3d55.jpg,Estudio de cabeza
0,404387d6-a52c-4477-b598-de2a2d5a3d55.jpg,Mitología
0,404387d6-a52c-4477-b598-de2a2d5a3d55.jpg,1764
0,404387d6-a52c-4477-b598-de2a2d5a3d55.jpg,+
1,589ee4a3-28fa-4977-a84d-7326f5c9aeb3.jpg,Sanguina
1,589ee4a3-28fa-4977-a84d-7326f5c9aeb3.jpg,Papel agarbanzado
1,589ee4a3-28fa-4977-a84d-7326f5c9aeb3.jpg,Desnudo femenino


## Graph creation and visualisation

In [45]:
pages = defaultdict(int)

for piece in prada.groupby('work_id'):
  tags = list(piece[1]['work_tags'])

  for i in range(len(tags)):
    for j in range(i+1, len(tags)):
      pages[(tags[i], tags[j])] += 1

pages

defaultdict(int,
            {('Torre de la Parada', 'Óleo'): 49,
             ('Torre de la Parada', 'Lienzo'): 39,
             ('Torre de la Parada', 'Alegoría del Amor'): 1,
             ('Torre de la Parada', 'Cupido/Eros'): 2,
             ('Torre de la Parada', 'Candil'): 1,
             ('Torre de la Parada', '+'): 49,
             ('Óleo', 'Lienzo'): 3203,
             ('Óleo', 'Alegoría del Amor'): 7,
             ('Óleo', 'Cupido/Eros'): 15,
             ('Óleo', 'Candil'): 3,
             ('Óleo', '+'): 4027,
             ('Lienzo', 'Alegoría del Amor'): 7,
             ('Lienzo', 'Cupido/Eros'): 15,
             ('Lienzo', 'Candil'): 3,
             ('Lienzo', '+'): 3098,
             ('Alegoría del Amor', 'Cupido/Eros'): 4,
             ('Alegoría del Amor', 'Candil'): 1,
             ('Alegoría del Amor', '+'): 11,
             ('Cupido/Eros', 'Candil'): 1,
             ('Cupido/Eros', '+'): 51,
             ('Candil', '+'): 3,
             ('Esculpido', 'Mármol'): 273,


In [47]:
G = nx.Graph()

for page in pages:
    artist1, artist2 = page
    if artist1 != artist2:
      score = pages[page]

    # The edge is only created when the score is high enough -> at least 2 users have listened to the pair
      if score >= 1:
        G.add_edge(artist1, artist2, weight=score)

print("Total number of graph nodes:", G.number_of_nodes())
print("Total number of graph edges:", G.number_of_edges())

Total number of graph nodes: 3548
Total number of graph edges: 52846


Build the transition matrix (the base to perform PageRank)

In [50]:
G.edges(data=True)

EdgeDataView([('Torre de la Parada', 'Óleo', {'weight': 49}), ('Torre de la Parada', 'Lienzo', {'weight': 39}), ('Torre de la Parada', 'Alegoría del Amor', {'weight': 1}), ('Torre de la Parada', 'Cupido/Eros', {'weight': 2}), ('Torre de la Parada', 'Candil', {'weight': 1}), ('Torre de la Parada', '+', {'weight': 49}), ('Torre de la Parada', 'Tabla de madera de roble', {'weight': 1}), ('Torre de la Parada', 'Animales', {'weight': 2}), ('Torre de la Parada', 'Caza / Cazador', {'weight': 1}), ('Torre de la Parada', 'Desnudo femenino', {'weight': 6}), ('Torre de la Parada', 'Paisaje', {'weight': 2}), ('Torre de la Parada', 'Héroes y heroínas', {'weight': 14}), ('Torre de la Parada', 'Céfalo', {'weight': 2}), ('Torre de la Parada', 'Desnudo infantil', {'weight': 3}), ('Torre de la Parada', 'Dioses mitológicos', {'weight': 23}), ('Torre de la Parada', 'Tabla', {'weight': 9}), ('Torre de la Parada', 'Desnudo masculino', {'weight': 13}), ('Torre de la Parada', '1636', {'weight': 10}), ('Torre 

In [48]:
adjacency = {}
for u in range(len(pages)):
    adjacency[u] = []

for (a, b) in links:
    adjacency[a].append(b)

print(adjacency)